## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
pd.options.mode.chained_assignment = None  # default='warn'

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Surab,PK,28.4914,66.2585,93.99,2,0,18.63,clear sky
1,1,Forestville,US,38.8451,-76.8750,71.49,81,1,1.01,clear sky
2,2,Narsaq,GL,60.9167,-46.0500,40.23,75,79,3.51,broken clouds
3,3,East London,ZA,-33.0153,27.9116,70.74,60,0,4.61,clear sky
4,4,Huarmey,PE,-10.0681,-78.1522,65.14,86,16,2.13,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Harper,LR,4.3750,-7.7169,81.64,78,65,7.61,light rain
10,10,Sinnamary,GF,5.3833,-52.9500,81.12,79,57,4.63,broken clouds
15,15,Axim,GH,4.8699,-2.2405,83.28,72,95,6.69,overcast clouds
19,19,Oeiras,BR,-7.0253,-42.1311,83.39,48,39,10.25,scattered clouds
22,22,Quimper,FR,48.0000,-4.1000,80.67,53,0,4.61,clear sky
29,29,Saint-Philippe,RE,-21.3585,55.7679,78.39,68,77,16.22,broken clouds
34,34,Georgetown,MY,5.4112,100.3354,85.98,76,20,5.01,moderate rain
42,42,Faya,SA,18.3851,42.4509,86.11,7,25,11.99,scattered clouds
48,48,Puri,IN,19.8000,85.8500,83.37,77,96,17.87,moderate rain
53,53,Saint George,US,37.1041,-113.5841,79.39,18,1,1.99,clear sky


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                309
City                   309
Country                305
Lat                    309
Lng                    309
Max Temp               309
Humidity               309
Cloudiness             309
Wind Speed             309
Current Description    309
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)
len(preferred_cities_df)

305

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Harper,LR,81.64,light rain,4.3750,-7.7169,
10,Sinnamary,GF,81.12,broken clouds,5.3833,-52.9500,
15,Axim,GH,83.28,overcast clouds,4.8699,-2.2405,
19,Oeiras,BR,83.39,scattered clouds,-7.0253,-42.1311,
22,Quimper,FR,80.67,clear sky,48.0000,-4.1000,
29,Saint-Philippe,RE,78.39,broken clouds,-21.3585,55.7679,
34,Georgetown,MY,85.98,moderate rain,5.4112,100.3354,
42,Faya,SA,86.11,scattered clouds,18.3851,42.4509,
48,Puri,IN,83.37,moderate rain,19.8000,85.8500,
53,Saint George,US,79.39,clear sky,37.1041,-113.5841,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
# hotel_df.notnull().sum()
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Harper,LR,81.64,light rain,4.3750,-7.7169,Screensaver
10,Sinnamary,GF,81.12,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
15,Axim,GH,83.28,overcast clouds,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
19,Oeiras,BR,83.39,scattered clouds,-7.0253,-42.1311,Hotel SESC Oeiras
22,Quimper,FR,80.67,clear sky,48.0000,-4.1000,Hôtel Escale Oceania Quimper
...,...,...,...,...,...,...,...
700,Marrakesh,MA,80.67,clear sky,31.6315,-8.0083,Novotel Marrakech Hivernage
706,Carutapera,BR,82.99,broken clouds,-1.1950,-46.0200,Lidera
709,Remanso,BR,78.80,scattered clouds,-9.6179,-42.0831,Hotel Mangueira
711,San Andres,CO,75.22,light intensity drizzle,12.5847,-81.7006,Hotel Arena Blanca


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))